* [네이버 오픈 API 목록 - INTRO](https://developers.naver.com/products/intro/plan/plan.md)
* [애플리케이션 - NAVER Developers](https://developers.naver.com/apps/#/list)

In [ ]:
import os
import re
import requests
import json
import pandas as pd
from dotenv import load_dotenv

load_dotenv()  # 현재 프로젝트 폴더(또는 지정한 경로)에 있는 .env 파일 로드

# 네이버 개발자 센터에서 발급받은 값(.env 파일에 저장되어 있다고 가정)
client_id = os.getenv('NAVER_CLIENT_ID')
client_secret = os.getenv('NAVER_CLIENT_SECRET')

# 검색할 키워드
keyword = "생성형AI"

# 요청 URL (JSON 결과)
url = "https://openapi.naver.com/v1/search/news.json"

# 요청 파라미터
params = {
    "query": keyword,   # 검색어
    "display": 10,      # 한 번에 가져올 검색 결과 개수
    "start": 1,         # 검색 시작 위치(최대 1000)
    "sort": "sim"       # 정렬 옵션: sim (유사도순), date (날짜순)
}

# 요청 헤더
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

def remove_html_tags(text):
    """HTML 태그를 제거하는 간단한 함수"""
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

# GET 요청
response = requests.get(url, headers=headers, params=params)

# 응답 상태 코드 확인
if response.status_code == 200:
    data = response.json()  # JSON 형태로 변환

    # 'items' 키가 있는지 확인
    if 'items' in data:
        # 각 뉴스 기사 정보가 담긴 리스트
        items = data['items']

        # 데이터프레임 변환
        df = pd.DataFrame(items)

        # title과 description에 포함된 HTML 태그 제거
        if 'title' in df.columns:
            df['title'] = df['title'].apply(remove_html_tags)
        if 'description' in df.columns:
            df['description'] = df['description'].apply(remove_html_tags)

        # 변환된 데이터프레임 출력
        print(df)
    else:
        print("응답 데이터에 'items'가 없습니다. 구조를 확인하세요.")
else:
    print(f"Error Code: {response.status_code}")
    print(response.text)


In [ ]:
df